In [14]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np



In [15]:

# Load the dataset
data_dir = "./processed"
img_height, img_width = 224, 224  # specify the height and width to resize images
batch_size = 32

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 21600 files belonging to 27 classes.


In [21]:
# Get the class names and convert them to numerical labels
class_names = dataset.class_names
encoder = LabelEncoder()
labels = encoder.fit_transform(class_names)

# class label maps
class_label_map = {label: class_name for class_name, label in zip(class_names, labels)}
print(class_label_map)

{0: '0', 1: '1', 2: '10', 3: '11', 4: '12', 5: '13', 6: '14', 7: '15', 8: '16', 9: '17', 10: '18', 11: '19', 12: '2', 13: '20', 14: '21', 15: '22', 16: '23', 17: '24', 18: '25', 19: '26', 20: '3', 21: '4', 22: '5', 23: '6', 24: '7', 25: '8', 26: '9'}


In [19]:
import numpy as np

# Initialize empty lists to hold the images and labels
images = []
labels_list = []

# Iterate over the dataset
for image_batch, label_batch in dataset:
    # Convert the batch to numpy arrays and add them to the lists
    images.append(image_batch.numpy())
    labels_list.append(label_batch.numpy())

# Concatenate the batches
images = np.concatenate(images)
labels_list = np.concatenate(labels_list)

# Flatten the images
images = images.reshape(images.shape[0], -1)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels_list, test_size=0.2, random_state=42)

# print the class indices and class names
print("Class indices: ", encoder.transform(class_names))
print("Class names: ", class_names)

KeyboardInterrupt: 

In [5]:
# Train the Random Forest model
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

RandomForestClassifier()

In [9]:
# save the model to .keras file
# model.save("model.p")

# save the random forest model to a file
import pickle
filename = 'finalized_model.p'
pickle.dump(model, open(filename, 'wb'))


In [10]:
# Evaluate the model
y_pred = model.predict(x_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.9861111111111112


In [14]:
print(type(x_test))

<class 'numpy.ndarray'>


In [34]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False, min_detection_confidence=0.9, min_tracking_confidence=0.9
)

# function to create image with landmarks
def get_hand_landmarks_image(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # draw landmarks on blank 224x224 white image
    output_img = np.ones((224, 224, 3), np.uint8) * 255

    results = hands.process(image_rgb)

    mp_drawing = mp.solutions.drawing_utils

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                output_img,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(20,20,20), thickness=3, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(20,20,20), thickness=3, circle_radius=1),
            )
    return not results.multi_hand_landmarks == None, output_img

In [24]:
# test with static image /processed/0/0.jpg
import cv2
import numpy as np
import tensorflow as tf
import pickle

# load the model
filename = 'finalized_model.p'
model = pickle.load(open(filename, 'rb'))

# Load the image
image = cv2.imread("./processed/1/500.jpg")

pred = model.predict(image.reshape(1, -1))

print("Prediction: ", class_label_map[pred[0]])


Prediction:  1


In [35]:
# test using the webcam 
import cv2
import numpy as np
import tensorflow as tf
import pickle

# Load the model
filename = 'finalized_model.p'
model = pickle.load(open(filename, 'rb'))

# Load the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    ret, frame = cap.read()

    # Preprocess the frame to match the input size of the model and scale the pixel values
    frame = frame[:, 80:560]
    frame = cv2.resize(frame, (224, 224))

    # Resize the image
    is_hands_detected, landmarked_img = get_hand_landmarks_image(frame)

    # Flatten the image
    img = landmarked_img.flatten().reshape(1, -1)

    # Make a prediction
    pred = model.predict(img)
    class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'blank']

    print(pred)

    cv2.putText(landmarked_img, class_labels[int(class_label_map[pred[0]])], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("frame", landmarked_img)

    # Check if the user pressed the ESC key
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[4]
[4]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[2

In [4]:
cap.release()
